In [2]:
# Call the relevant python packages
import numpy as np
import pandas as pd
import glob, os                          # Operating system module and pathnames

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import ascii, fits
from astropy.table import join, hstack, Table, Column, vstack
from astropy.coordinates import Angle, Latitude, Longitude, SkyCoord

#import SED Fitter module
from sedfitter import fit
from sedfitter import plot
from sedfitter.extinction import Extinction

# Set colour palette & plot layouts
import seaborn as sns     #; sns.set()
sns.set_context("paper")   # "paper", "notebook", "talk", "poster"

# Output plots in Notebook
#%matplotlib notebook
%matplotlib inline

In [5]:
# Read in the catalogue

file = '/scratch/s1625982/NGC6822/env/NGC6822_YSOs.csv'
data = ascii.read(file, data_start=1)

YSO_num=len(data)
print("The number of YSOs in the table is",YSO_num)

#set up new columns for the flux
data['J flux']=0.
data['H flux']=0.
data['K flux']=0.
data['36 flux']=0.
data['45 flux']=0.
data['58 flux']=0.
data['80 flux']=0.
data['24 flux']=0.

#set up new columns for the flux errors
data['J flux err']=0.
data['H flux err']=0.
data['K flux err']=0.
data['36 flux err']=0.
data['45 flux err']=0.
data['58 flux err']=0.
data['80 flux err']=0.
data['24 flux err']=0.

#NB: this program assumes sources are either valid or upper limits
#Hence, flags are either 1 (valid) or 3 (upper limit).
#Assume all sources to be valid initially
data['J int']=1
data['H int']=1
data['K int']=1
data['36 int']=1
data['45 int']=1
data['58 int']=1
data['80 int']=1
data['24 int']=1

#hardcode zero points of filters for Vega system
#fluxes given in mJy
F_0_J=1601000
m_0_J=-0.016
F_0_H=1034000
m_0_H=0.018
F_0_K=670300
m_0_K=0.008
F_0_36=280900
m_0_36=0
F_0_45=179700
m_0_45=0
F_0_58=115000
m_0_58=0
F_0_80=1034000
m_0_80=0
F_0_24=64900
m_0_24=0



#assign the integer describing source reliability
def write_column(idx,name):
    for i in range(len(idx[0])):
        #if source is upper limit write 3 flag
        data[name][idx[0][i]]=3

#create index to find which sources are upper limits
idx_J=np.where((data['J data?']==0))
write_column(idx_J,'J int')
idx_H=np.where((data['H data?']==0))
write_column(idx_H,'H int')
idx_K=np.where((data['J data?']==0))
write_column(idx_K,'K int')
idx_36=np.where((data['36 data?']==0))
write_column(idx_36,'36 int')
idx_45=np.where((data['45 data?']==0))
write_column(idx_45,'45 int')
idx_58=np.where((data['58 data?']==0))
write_column(idx_58,'58 int')
idx_80=np.where((data['80 data?']==0))
write_column(idx_80,'80 int')
idx_24=np.where((data['24 data?']==0))
write_column(idx_24,'24 int')


        
data['J flux']=F_0_J*np.power(10,0.4*(m_0_J-data['Jmag']))
data['H flux']=F_0_H*np.power(10,0.4*(m_0_H-data['Hmag']))
data['K flux']=F_0_K*np.power(10,0.4*(m_0_K-data['Kmag']))
data['36 flux']=F_0_36*np.power(10,0.4*(m_0_36-data['k36mag']))
data['45 flux']=F_0_45*np.power(10,0.4*(m_0_45-data['k45mag']))
data['58 flux']=F_0_58*np.power(10,0.4*(m_0_58-data['k58mag']))
data['80 flux']=F_0_80*np.power(10,0.4*(m_0_80-data['k80mag']))
data['24 flux']=F_0_24*np.power(10,0.4*(m_0_24-data['k24mag']))
#data.show_in_browser(jsviewer=True)

ascii.NoHeader()

#write out table in the right format for SED fitter
ascii.write(data[['New name','RA','Dec','J int','H int','K int','36 int','45 int','58 int','80 int','24 int',
                  'J flux','J flux err','H flux','H flux err','K flux','K flux err','36 flux','36 flux err',
                  '45 flux','45 flux err','58 flux','58 flux err','80 flux','80 flux err','24 flux','24 flux err']],
            'NGC6822_YSOs_SED_format.csv', format='no_header', fast_writer=False)

The number of YSOs in the table is 584


In [ ]:
# Define path to models
model_dir = '/scratch/s1625982/NGC6822/env/environ/models_r06'

# Read in extinction law)
extinction = Extinction.from_file('/scratch/s1625982/NGC6822/env/environ/sedfitter-1.3/examples/kmh94.par',
                                  columns=[0, 3], wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Define filters and apertures
filters = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
#The actual aperture sizes for these filters can be found in our paper
apertures = [3., 3., 3., 3., 3., 3., 3., 3.] * u.arcsec

# Run the fitting
SED_input_file = '/scratch/s1625982/NGC6822/env/environ/NGC6822_YSOs_SED_format_use_0.csv'
data = ascii.read(SED_input_file, data_start=1)
fit(SED_input_file, filters, apertures, model_dir,
    'output_use_0.fitinfo',
    extinction_law=extinction,
    distance_range=[450., 530.] * u.kpc,
    av_range=[0., 5.])

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  YSO models
   Log[d] stepping     :  0.01
   Number of distances :  8

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2J.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2H.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/2K.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I1.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I2.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I3.fits
   Reading /scratch/s1625982/NGC6822/env/environ/models_r06/convolved/I4.fits
   Reading /scratch/s1625982/NGC6822/env/environ/mod

In [4]:
plot('/scratch/s1625982/NGC6822/env/environ/output_use_0.fitinfo', 'plots_seds_use_0')

Delete directory plots_seds_use_0? [y/[n]] y

